In [156]:
__file__ = "__init__.py"

In [157]:
import sys, os, json5, glob
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration, T5Tokenizer



project_root = Path(__file__).resolve().parents[1]
sys.path.append(str(project_root))

from utils.qgene import generate_text

paths = {
    "processed": os.path.abspath(f"{project_root}/data/storage/processed"),
    "qfragments": os.path.abspath(f"{project_root}/intents/qfragments.json"),
    "questions": os.path.abspath(f"{project_root}/intents/questions.csv"),
    "qtrain": os.path.abspath(f"{project_root}/intents/qtrain.csv"),
    "models": os.path.abspath(f"{project_root}/models/t5-small"),
    "results": os.path.abspath(f"{project_root}/training/results"),
}

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        
tf.device('/device:GPU:0')

Physical devices cannot be modified after being initialized


In [158]:
def dataset(datanum: int = 1000) -> pd.DataFrame:
    """
    Sinh dữ liệu từ generate_text với choice="s1", 
    kết hợp dữ liệu cũ (nếu có) và lưu vào file qtrain.csv.
    """
    if os.path.exists(paths["qtrain"]):
        old_data = pd.read_csv(paths["qtrain"])
        new_questions = pd.concat([old_data, generate_text(datanum, choice="s1")])
    else:
        new_questions = generate_text(datanum, choice="s1")
    new_questions.to_csv(paths["qtrain"], index=False)
    return new_questions

def load_dataset(num_samples=10000, batch_size=32, shuffle_buffer=1000, split_ratio=0.1):
    """
    Sinh dữ liệu bằng hàm dataset, chuyển đổi sang tf.data.Dataset ở dạng raw,
    tách dữ liệu thành train và validation trước khi batching.
    
    Args:
        num_samples: Số mẫu dữ liệu cần sinh.
        batch_size: Kích thước batch.
        shuffle_buffer: Kích thước buffer cho việc shuffle.
        split_ratio: Tỷ lệ dữ liệu dùng cho validation (mặc định 0.1).
        
    Returns:
        Một tuple (train_dataset, val_dataset), mỗi dataset đã được batch và prefetch.
    """
    df = dataset(num_samples)
    
    # Tạo cột input: thêm "Extract component: " vào câu hỏi
    inputs = df["question"].apply(lambda x: "Extract component: " + x).tolist()
    
    # Tạo cột target: chuỗi chứa các component theo định dạng cố định
    targets = df.apply(
        lambda row: (
            f"BRAND: {row['brand']}; "
            f"PRICE: {row['price']}; "
            f"RAM: {row['ram']}; "
            f"GPU: {row['gpu']}; "
            f"CPU: {row['cpu']}; "
            f"DISPLAY: {row['display']}; "
            f"REFRESH_RATE: {row['refresh rate']}"
        ).strip(),
        axis=1
    ).tolist()
    
    # Tạo raw dataset chưa batch
    raw_dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
    raw_dataset = raw_dataset.shuffle(shuffle_buffer)
    
    total_samples = len(inputs)
    val_size = int(total_samples * split_ratio)
    
    # Tách dữ liệu: Lấy val_size mẫu cho validation, phần còn lại cho training
    val_dataset = raw_dataset.take(val_size)
    train_dataset = raw_dataset.skip(val_size)
    
    # Sau đó, áp dụng batch và prefetch cho cả train và validation
    train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return train_dataset, val_dataset

In [159]:
train_ds, val_ds = load_dataset(num_samples=100, batch_size=4)
print("Train dataset:")
for inp, tgt in train_ds.take(1):
    print("Input mẫu:", inp.numpy()[0])
    print("Target mẫu:", tgt.numpy()[0])
print("\nValidation dataset:")
for inp, tgt in val_ds.take(1):
    print("Input mẫu:", inp.numpy()[0])
    print("Target mẫu:", tgt.numpy()[0])

Train dataset:
Input mẫu: b'Extract component: Assistance in finding laptops engineered by lenovo, equipped with intel core i7 9750h besides nvidia rtx a3000 plus ram 32gb in addition to 3k2k display panel alongside 300hz ideal for running machine learning frameworks above 1688 USD.'
Target mẫu: b'BRAND: lenovo; PRICE: above 1688 USD; RAM: ram 32gb; GPU: nvidia rtx a3000; CPU: intel core i7 9750h; DISPLAY: 3k2k display panel; REFRESH_RATE: 300hz'

Validation dataset:
Input mẫu: b'Extract component: Show me some laptops crafted by dell, that have ryzen ai 7 & amd radeon rx 6800s coupled with 24gb ram as well as display resolution 4k accompanied by 240hz suitable for running high-performance data modeling software in the region of $2972.'
Target mẫu: b'BRAND: dell; PRICE: in the region of $2972; RAM: 24gb ram; GPU: amd radeon rx 6800s; CPU: ryzen ai 7; DISPLAY: display resolution 4k; REFRESH_RATE: 240hz'


In [160]:
def build_projection_layer(input_dim=256, output_dim=512):
    """
    Xây dựng một projection layer dùng tf.keras.layers.Dense chuyển từ 256 lên 512.
    """
    projection_layer = tf.keras.layers.Dense(output_dim, input_shape=(input_dim,))
    return projection_layer

vocab_size = 10
embedding_matrix = np.random.randn(vocab_size, 256).astype("float32")
projection = build_projection_layer(256, 512)
projected = projection(embedding_matrix)
print("Kích thước sau projection:", projected.shape)  # (10, 512)

/home/onsra/miniconda3/envs/Astorine/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Kích thước sau projection: (10, 512)


In [161]:
def integrate_custom_embeddings(embedding_matrix, projection_layer, t5_model, union_vocab):
    """
    Cập nhật embedding layer của T5 với custom embeddings sau khi qua projection layer.
    
    Args:
        embedding_matrix: numpy array (vocab_size, 256)
        projection_layer: tf.keras.layers.Dense chuyển từ 256 -> 512
        t5_model: mô hình TFT5ForConditionalGeneration
        union_vocab: dict mapping token -> index (vocabulary)
    
    Returns:
        t5_model với embedding đã được cập nhật.
    """
    # Resize embedding của T5 cho phù hợp với union_vocab mới.
    new_vocab_size = len(union_vocab)
    t5_model.resize_token_embeddings(new_vocab_size)
    
    # Chuyển đổi embedding_matrix sang tensor và qua projection layer.
    custom_embeddings = tf.convert_to_tensor(embedding_matrix, dtype=tf.float32)  # (vocab_size, 256)
    projected_embeddings = projection_layer(custom_embeddings)  # (vocab_size, 512)
    
    # Cập nhật embedding layer (shared) của T5 sử dụng set_weights.
    # Lưu ý: set_weights cần nhận vào danh sách các mảng numpy.
    t5_model.shared.set_weights([projected_embeddings.numpy()])
    
    # Đảm bảo embedding layer được fine-tune (trainable).
    t5_model.shared.trainable = True
    return t5_model

In [162]:
def get_checkpoint_name():
    """
    Trả về tên checkpoint theo định dạng: checkpoint-{HHMM}-{DDMMYYYY}
    Giờ ở định dạng 24h.
    """
    now = datetime.datetime.now()
    checkpoint_name = f"checkpoint-{now.strftime('%H%M')}-{now.strftime('%d%m%Y')}"
    return checkpoint_name

class CustomModelCheckpoint(tf.keras.callbacks.Callback):
    """
    Callback lưu model ở định dạng SavedModel sau mỗi epoch và
    tự động xóa các checkpoint cũ nếu số lượng vượt quá 3.
    """
    def __init__(self, monitor='val_loss', mode='min', early_stop=5):
        super(CustomModelCheckpoint, self).__init__()
        self.monitor = monitor
        self.mode = mode
        self.early_stop = early_stop  # Nếu early_stop là False thì tắt EarlyStopping, nếu int thì dùng giá trị đó.
        self.checkpoints_dir = paths["results"]
        os.makedirs(self.checkpoints_dir, exist_ok=True)
        self.checkpoints = []
    
    def on_epoch_end(self, epoch, logs=None):
        # Lưu model sau mỗi epoch.
        checkpoint_name = get_checkpoint_name()
        save_path = os.path.join(self.checkpoints_dir, f"{checkpoint_name}")
        self.model.save(save_path, save_format="tf")
        self.checkpoints.append(save_path)
        print(f"Đã lưu checkpoint: {save_path}")
        self._cleanup_checkpoints()
    
    def _cleanup_checkpoints(self):
        # Xóa các checkpoint cũ nếu số lượng vượt quá 3.
        if len(self.checkpoints) > 3:
            num_to_delete = len(self.checkpoints) - 3
            for i in range(num_to_delete):
                checkpoint_to_delete = self.checkpoints.pop(0)
                # Xóa thư mục checkpoint
                tf.io.gfile.rmtree(checkpoint_to_delete)
                print(f"Đã xóa checkpoint cũ: {checkpoint_to_delete}")

def get_callbacks(monitor='val_loss', mode='min', early_stop=True):
    """
    Trả về danh sách các callback cho việc huấn luyện.
    early_stop: Nếu là True (hoặc số int), sử dụng EarlyStopping với patience=5 (hoặc giá trị int đó).
                Nếu là False, không sử dụng EarlyStopping.
    """
    callbacks = []
    # Callback lưu checkpoint
    checkpoint_cb = CustomModelCheckpoint(monitor=monitor, mode=mode, early_stop=5 if early_stop is True else early_stop)
    callbacks.append(checkpoint_cb)
    
    # Callback EarlyStopping
    if early_stop:
        patience = 5 if early_stop is True else early_stop
        early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor=monitor, mode=mode, patience=patience)
        callbacks.append(early_stop_cb)
    
    return callbacks


cbs = get_callbacks()
print("Số callback:", len(cbs))

Số callback: 2


In [163]:
global t5_tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

def tf_tokenize_map(input_text, target_text):
    input_ids, attention_mask, labels = tf.py_function(
        func=tokenize_function,
        inp=[input_text, target_text],
        Tout=(tf.int32, tf.int32, tf.int32)
    )
    input_ids.set_shape([None, 512])
    attention_mask.set_shape([None, 512])
    labels.set_shape([None, 128])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

def tokenize_function(input_text, target_text):
    # Chuyển đổi tensor thành danh sách (list)
    input_list = input_text.numpy().tolist()
    target_list = target_text.numpy().tolist()
    
    # Giải mã từng phần tử nếu cần (nếu là bytes)
    input_list = [s.decode('utf-8') if isinstance(s, bytes) else s for s in input_list]
    target_list = [s.decode('utf-8') if isinstance(s, bytes) else s for s in target_list]
    
    tokenized_inputs = t5_tokenizer(
        input_list,
        padding='max_length',
        truncation=True,
        max_length=256,
        return_tensors="np"
    )
    tokenized_targets = t5_tokenizer(
        target_list,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="np"
    )
    return tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'], tokenized_targets['input_ids']

def custom_train_loop(model, train_dataset, val_dataset, optimizer, num_epochs=10, accumulation_steps=4):
    """
    Huấn luyện mô hình với gradient accumulation.
    
    Args:
      model: mô hình T5 đã tích hợp custom embeddings.
      train_dataset: tf.data.Dataset đã token hóa cho training.
      val_dataset: tf.data.Dataset đã token hóa cho validation.
      optimizer: Optimizer (ví dụ, Adam).
      num_epochs: Số epoch huấn luyện.
      accumulation_steps: Số bước tích lũy gradient trước khi cập nhật trọng số.
    """
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        epoch_loss = 0.0
        batch_count = 0
        # Khởi tạo danh sách tích lũy gradient cho từng biến trainable
        accumulated_gradients = [tf.zeros_like(var) for var in model.trainable_variables]
        
        for step, batch in enumerate(train_dataset):
            # Mỗi batch đã ở dạng dictionary: {'input_ids':..., 'attention_mask':..., 'labels':...}
            with tf.GradientTape() as tape:
                outputs = model(
                    input_ids=batch['input_ids'], 
                    attention_mask=batch['attention_mask'], 
                    labels=batch['labels'], 
                    training=True
                )
                # Chia loss cho số bước tích lũy để cân bằng
                loss = outputs.loss / accumulation_steps
            gradients = tape.gradient(loss, model.trainable_variables)
            # Tích lũy gradient
            accumulated_gradients = [accum_grad + grad for accum_grad, grad in zip(accumulated_gradients, gradients)]
            epoch_loss += outputs.loss
            batch_count += 1
            
            if (step + 1) % accumulation_steps == 0:
                optimizer.apply_gradients(zip(accumulated_gradients, model.trainable_variables))
                # Reset accumulated gradients
                accumulated_gradients = [tf.zeros_like(var) for var in model.trainable_variables]
                print(f"Step {step+1}: Loss = {(loss * accumulation_steps).numpy():.4f}")
        
        avg_epoch_loss = epoch_loss / batch_count
        print(f"Epoch {epoch+1} Average Training Loss: {avg_epoch_loss:.4f}")
        
        # Validation sau mỗi epoch
        val_loss = 0.0
        val_batches = 0
        for batch in val_dataset:
            outputs = model(
                input_ids=batch['input_ids'], 
                attention_mask=batch['attention_mask'], 
                labels=batch['labels'], 
                training=False
            )
            val_loss += outputs.loss
            val_batches += 1
        avg_val_loss = val_loss / val_batches if val_batches > 0 else 0
        print(f"Epoch {epoch+1} Validation Loss: {avg_val_loss:.4f}")

def main():
    # 1. Load raw dataset: train and validation
    train_raw, val_raw = load_dataset(num_samples=10000, batch_size=32, split_ratio=0.1)
    
    # 2. Tải tokenizer T5
    t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
    
    # 3. Xây dựng union vocabulary (ở đây dùng vocabulary của tokenizer T5)
    union_vocab = t5_tokenizer.get_vocab()
    print("Vocabulary size:", len(union_vocab))
    
    # 4. Tạo embedding matrix giả lập (với shape: vocab_size x 256)
    vocab_size = len(union_vocab)
    embedding_matrix = np.random.randn(vocab_size, 256).astype("float32")
    
    # 5. Xây dựng projection layer (256 -> 512)
    projection_layer = build_projection_layer(input_dim=256, output_dim=512)
    
    # 6. Tải mô hình T5 phiên bản TensorFlow
    t5_model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
    
    # 7. Tích hợp custom embeddings vào T5
    t5_model = integrate_custom_embeddings(embedding_matrix, projection_layer, t5_model, union_vocab)
    print("Custom embeddings đã được tích hợp vào T5.")
    
    # 8. Áp dụng hàm token hóa cho train và validation dataset
    train_dataset = train_raw.map(tf_tokenize_map, num_parallel_calls=tf.data.AUTOTUNE)
    val_dataset = val_raw.map(tf_tokenize_map, num_parallel_calls=tf.data.AUTOTUNE)
    
    # 9. Thiết lập optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    
    # 10. Chạy custom training loop với gradient accumulation
    custom_train_loop(t5_model, train_dataset, val_dataset, optimizer, num_epochs=10, accumulation_steps=4)
    
    # 11. Ví dụ inference: Lấy một batch từ val_dataset và generate output
    for batch in val_dataset.take(1):
        outputs = t5_model.generate(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            max_length=128
        )
        decoded_outputs = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print("Inference Output mẫu:", decoded_outputs)
        break
